In [ ]:
population_a = populations[0]
population_b = populations[1]
population_a_target = 4
population_b_target = 2

temp_a_cross = {}
temp_b_cross = {}

population_a_temp = deepcopy(population_a)
population_b_temp = deepcopy(population_b)

for i in population_a_temp.get_partition_points(population_a_target):
    partition = population_b.get_points().get(i).get('partition')
    val = temp_a_cross.get(partition, 0)
    temp_a_cross[partition] = val + 1

for i in population_b.get_partition_points(max(temp_a_cross, key=temp_a_cross.get)):
    population_b.update_partition(i, 0)
    
for i in population_a_temp.get_partition_points(population_a_target):
    population_b.update_partition(i, max(temp_a_cross, key=temp_a_cross.get))

for i in population_b_temp.get_partition_points(population_b_target):
    partition = population_a.get_points().get(i).get('partition')
    val = temp_b_cross.get(partition, 0)
    temp_b_cross[partition] = val + 1

for i in population_a.get_partition_points(max(temp_b_cross, key=temp_b_cross.get)):
    population_a.update_partition(i, 0)
    
for i in population_b_temp.get_partition_points(population_b_target):
    population_a.update_partition(i, max(temp_a_cross, key=temp_a_cross.get))


In [ ]:
G = nx.Graph(np.array(population_a.weight))

a_check = False 
b_check = False 

while len(population_a.get_partition_points(0)) > 0 or len(population_b.get_partition_points(0)) > 0:
    population_a_excess = population_a.get_partition_points(0)
    population_b_excess = population_b.get_partition_points(0)
    
    count_null_population_a = 0
    for i in population_a_excess:
        partition_id, min_partition = -1, -1
        for j in [n for n in G.neighbors(i)]:
            partition = population_a.get_points().get(j).get('partition')
            if(min_partition == -1 and (partition != 0 and partition != max(temp_a_cross, key=temp_a_cross.get))):
                partition_id, min_partition = partition, len(population_a.get_partition_points(partition))
            elif(min_partition > len(population_a.get_partition_points(partition)) and (partition != 0 and partition != max(temp_a_cross, key=temp_a_cross.get))):
                partition_id, min_partition = partition, len(population_a.get_partition_points(partition))
        if(a_check):
            population_a.update_partition(i, max(temp_a_cross, key=temp_a_cross.get))
        elif(min_partition != -1):
            population_a.update_partition(i, partition_id)
        elif(min_partition == -1):
            count_null_population_a += 1

    if(count_null_population_a == len(population_a_excess)):
        a_check = True

    count_null_population_b = 0
    for i in population_b_excess:
        partition_id, min_partition = -1, -1
        for j in [n for n in G.neighbors(i)]:
            partition = population_b.get_points().get(j).get('partition')
            if(min_partition == -1 and (partition != 0 and partition != max(temp_b_cross, key=temp_b_cross.get))):
                partition_id, min_partition = partition, len(population_b.get_partition_points(partition))
            elif(min_partition > len(population_b.get_partition_points(partition)) and (partition != 0 and partition != max(temp_b_cross, key=temp_b_cross.get))):
                partition_id, min_partition = partition, len(population_b.get_partition_points(partition))
        if(b_check):
            population_b.update_partition(i, max(temp_b_cross, key=temp_b_cross.get))
        elif(min_partition != -1):
            population_b.update_partition(i, partition_id)
        elif(min_partition == -1):
            count_null_population_b += 1

    if(count_null_population_b == len(population_b_excess)):
        b_check = True
